In [ ]:
# pip install --user --upgrade git+https://github.com/himanshudabas/twint.git@origin/twint-fixes#egg=twint

In [ ]:
import numpy as np
import pandas as pd
import twint
import time
# from datetime import datetime
from tqdm import tqdm

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

# PATHS

In [ ]:
split = 

In [ ]:
READ_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Mikhael Splits for Searching/Split {split}.csv"

SAVE_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Searched User Results/Split {split}.csv"

# Import Split List

In [ ]:
local_split_users = pd.read_csv(READ_SPLIT_PATH)

In [ ]:
# local_split_users

# Define Lookup Function

In [ ]:
cols_to_return = ['location', 'following', 'followers', 'likes', 'tweets', 'private', 'verified', 'join_date']

In [ ]:
def get_user_info(ID):
    c = twint.Config()
    c.User_id = ID
    c.Hide_output = True
    c.Pandas = True
    
    twint.run.Lookup(c)
    return twint.storage.panda.User_df[cols_to_return].values.tolist()[0]

# Apply Lookup Function

In [ ]:
# DELETE THIS
# local_split_users = MASTER_USER_LIST.head()

In [ ]:
user_lookup_dict = {}

for ID in local_split_users['ID'].astype(str).tolist():
    
    user_lookup_dict[ID] = get_user_info(ID)

# Merge Results with Split

## Create DataFrame of get_user_info() output

In [ ]:
user_information = pd.DataFrame.from_dict(user_lookup_dict, orient='index', columns=cols_to_return)

user_information = user_information.reset_index().rename(columns={'index':'ID'})

user_information['ID'] = pd.to_numeric(user_information['ID'])

## Merge DataFrame of get_user_info() output with local_split_users (on 'ID')

In [ ]:
local_split_users = local_split_users.merge(user_information, how='left', on='ID')

In [ ]:
local_split_users

# Export Results

In [ ]:
local_split_users.to_csv(SAVE_SPLIT_PATH, index=False)

# Split Users for Lookup

In [ ]:
MASTER_USER_LIST = pd.read_csv(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/Master User List - GEOLOCATED.csv")

# MASTER_USER_LIST[~MASTER_USER_LIST['FIPS - ELECTION 2020'].isnull()]['Num CT Tweets - HT'].hist(bins=100, log=True)

MIKHAEL_SPLIT_SAVE_PATH_ROOT = r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Mikhael Splits for Searching/"
LUKE_SPLIT_SAVE_PATH_ROOT = r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Lukes Splits for Searching/"

NUM_SPLITS = 20

for split in range(0, NUM_SPLITS):
    if split <= 9:
        MASTER_USER_LIST[split::NUM_SPLITS].to_csv(MIKHAEL_SPLIT_SAVE_PATH_ROOT + f'Split {split}.csv', index=False)
    else:
        MASTER_USER_LIST[split::NUM_SPLITS].to_csv(LUKE_SPLIT_SAVE_PATH_ROOT + f'Split {split}.csv', index=False)